In [1]:
import re
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
html_element = re.compile(r'<.*?>')

**Getting Urls**

In [3]:
All_links = []

for i in range(1,41):
    Link = f"https://lawtimesjournal.in/category/case-summary/page/{i}/"
    All_links.append(Link)

In [4]:
URLs = []

for i, link_ in enumerate(All_links):
    
    data = requests.get(link_)
    soup = BeautifulSoup(data.content, "html.parser")
    for i in soup.find_all('div', class_ = "td-block-span6"):
        link = i.find('a').attrs.get('href')
    
        URLs.append(link)

In [5]:
len(URLs)

400

**Getting Data**

In [30]:
titles_all=[]
summaries_all=[]
links_all=[]

for i,link in enumerate(URLs):
  data=requests.get(link)
  soup = BeautifulSoup(data.content,'lxml')

  try:
    title_tag=soup.find('h1',class_="entry-title")
    title=title_tag.text

  except Exception as e:
    title = np.NaN
    print(f"Missing title in URL {link}")

  try:
    summary_tag=soup.find('div',class_="td-post-content tagdiv-type").find_all('p')
    summary=" ".join([html_element.sub("",str(i)) for i in summary_tag])
    # link_content=summary[summary.rfind('References: '):]
    summary = summary[:summary.rfind('References: ')]
  except Exception as e:
    summary = np.NaN
    print(f"Missing summary in URL {link}")

  try:
    link_tag=soup.find('div',class_="td-post-content tagdiv-type").find('a')['href']
    print(f"{link_tag},{link}")
  except Exception as e:
    try:
      link_tag=soup.find('div',class_="wp-block-embed__wrapper").text.strip()
      print(f"{link_tag},{link}")
    except:
      try:
        
        link_content=soup.find('div',class_="td-post-content tagdiv-type").text
        link_content=link_content[link_content.rfind('References:'): ].strip()    
        
        if link_content.rfind('https://indiankanoon.org/doc/')==-1:
          link_tag=np.NaN
          print(f"Missing link in URL {link}")

        else:
          link_tag=link_content[link_content.rfind('https://indiankanoon.org/doc/'):link_content.rfind("")].strip()
          print(f"{link_tag},{link}")


      except Exception as e:
        link_tag=np.NaN
        print(f"Missing link in URL {link}")

  titles_all.append(title)
  summaries_all.append(summary)
  links_all.append(link_tag)

https://indiankanoon.org/doc/1978263/,https://lawtimesjournal.in/anand-sarup-sharma-vs-p-p-khurana-and-others/
https://indiankanoon.org/doc/143531/,https://lawtimesjournal.in/indira-khurana-vs-prem-prakash/
http://www.indiankanoon.com,https://lawtimesjournal.in/bhola-ram-sahu-vs-state-of-chhattisgarh/
https://indiankanoon.org/doc/59542/,https://lawtimesjournal.in/united-india-fire-general-insurance-company-limited-vs-smt-kanchanbai-ors/
https://caselaw.in/supreme-court/framing-charge-madhya-pradesh-deepak/4547/,https://lawtimesjournal.in/state-of-madhya-pradesh-vs-deepak/
https://indiankanoon.org/doc/677388/,https://lawtimesjournal.in/the-state-of-maharashtra-vs-syndicate-transport-company-pvt-ltd/
https://casetext.com,https://lawtimesjournal.in/neal-v-city-of-morrilton/
https://indiankanoon.org/doc/723973/,https://lawtimesjournal.in/poonam-verma-vs-ashwin-patel-1996-4-scc-332/
Missing link in URL https://lawtimesjournal.in/impact-of-toyota-prius-case-on-the-issue-determination-of-tran

In [31]:
Final_df = pd.DataFrame(list(zip(titles_all,  summaries_all, links_all)), columns =['titles', 'summaries','links'])
Final_df

,titles,summaries,links
0,Anand Sarup Sharma vs P.P. Khurana and Others,Introduction: The court in the present case de...,https://indiankanoon.org/doc/1978263/
1,Indira Khurana vs. Prem Prakash,Introduction: Parents play a significant role...,https://indiankanoon.org/doc/143531/
2,Bhola Ram Sahu vs State Of Chhattisgarh,Introduction Challenge in this appeal is to th...,http://www.indiankanoon.com
3,United India Fire & General Insurance Company ...,Introduction: Every contract is an agreement b...,https://indiankanoon.org/doc/59542/
4,State of Madhya Pradesh vs. Deepak,Introduction: In India an attempt to commit s...,https://caselaw.in/supreme-court/framing-charg...
...,...,...,...
395,Minerva Mills Vs. Union of India – Case Summary,The Parliament in order to serve general publi...,//lawtimesjournal.in/indira-nehru-gandhi-v-raj...
396,Indira Nehru Gandhi Vs. Raj Narain – Case Summary,Background Raj Narain was the political conten...,//lawtimesjournal.in/i-c-golaknath-ors-vs-stat...
397,I. C. Golaknath & Ors. Vs. State of Punjab & A...,Background Whether parliament can amend Fundam...,//lawtimesjournal.in/kesavananda-bharti-vs-sta...
398,Kesavananda Bharati Vs. State of Kerala – Case...,Background Facts His Holiness SripadGalvaru Ke...,//lawtimesjournal.in/i-c-golaknath-ors-vs-stat...


In [32]:
missing_values = Final_df.isna().sum()
percentage_missing = missing_values/Final_df.shape[0]
pd.concat([missing_values,percentage_missing],axis=1,keys=["Missing Values","% Missing"]).sort_values(by="% Missing",ascending=False)

,Missing Values,% Missing
links,59,0.1475
titles,0,0.0000
summaries,0,0.0000


In [35]:
Final_df = Final_df[Final_df["links"].isnull()==False]
missing_values = Final_df.isna().sum()
percentage_missing = missing_values/Final_df.shape[0]
pd.concat([missing_values,percentage_missing],axis=1,keys=["Missing Values","% Missing"]).sort_values(by="% Missing",ascending=False)

,Missing Values,% Missing
titles,0,0.0
summaries,0,0.0
links,0,0.0


In [36]:
import validators
Final_df['isURL'] = Final_df['links'].apply(lambda x : True if validators.url(x) else False)
Final_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,titles,summaries,links,isURL
0,Anand Sarup Sharma vs P.P. Khurana and Others,Introduction: The court in the present case de...,https://indiankanoon.org/doc/1978263/,True
1,Indira Khurana vs. Prem Prakash,Introduction: Parents play a significant role...,https://indiankanoon.org/doc/143531/,True
2,Bhola Ram Sahu vs State Of Chhattisgarh,Introduction Challenge in this appeal is to th...,http://www.indiankanoon.com,True
3,United India Fire & General Insurance Company ...,Introduction: Every contract is an agreement b...,https://indiankanoon.org/doc/59542/,True
4,State of Madhya Pradesh vs. Deepak,Introduction: In India an attempt to commit s...,https://caselaw.in/supreme-court/framing-charg...,True
...,...,...,...,...
336,Minerva Mills Vs. Union of India – Case Summary,The Parliament in order to serve general publi...,//lawtimesjournal.in/indira-nehru-gandhi-v-raj...,False
337,Indira Nehru Gandhi Vs. Raj Narain – Case Summary,Background Raj Narain was the political conten...,//lawtimesjournal.in/i-c-golaknath-ors-vs-stat...,False
338,I. C. Golaknath & Ors. Vs. State of Punjab & A...,Background Whether parliament can amend Fundam...,//lawtimesjournal.in/kesavananda-bharti-vs-sta...,False
339,Kesavananda Bharati Vs. State of Kerala – Case...,Background Facts His Holiness SripadGalvaru Ke...,//lawtimesjournal.in/i-c-golaknath-ors-vs-stat...,False


In [38]:
Final_df[(Final_df.links=='#_edn1')|(Final_df.links=='#_ftn1')|(Final_df.links=='about:blank')]

,titles,summaries,links,isURL
49,All India Bank Employees’ Association vs. Nati...,The provisions of the Companies Act of 1913 un...,about:blank,False
57,Suchita Srivastava vs Chandigarh Administration,In the Supreme Court of India(2009) 14 SCR 989...,about:blank,False
63,V. Jayakumar vs. The High Court of Kerala & ors.,The petitioner was compulsorily retired under ...,about:blank,False
64,Abhijeet.J.K vs. State of Kerala and others,"On 12.07.2018, at about 21.30 hours, the victi...",about:blank,False
79,P. Ulaganathan vs. The Government of India,Facts of the Case: The petitioner is the desce...,about:blank,False
96,Shalu Ojha vs. Prashant Ojha,The dispute is related to the rate of maintena...,about:blank,False
102,Wassan Singh vs. The State of Punjab,The landmark case dwells on the preventive rig...,about:blank,False
108,Suchitra Kumar Singha Roy vs. Smt. Arpita Sing...,The petitioner/husband was married to the re...,about:blank,False
113,John K. Illikkadan vs. Union of India & Ors.,Petitioner was appointed as a District Judge...,about:blank,False
137,Federation of Obstetric and Gynaecological Soc...,The Petitioner Federation of Obstetrics and ...,about:blank,False


In [41]:
Final_df=Final_df[Final_df['isURL']]
Final_df.reset_index(inplace=True,drop=True)
Final_df

,titles,summaries,links,isURL
0,Anand Sarup Sharma vs P.P. Khurana and Others,Introduction: The court in the present case de...,https://indiankanoon.org/doc/1978263/,True
1,Indira Khurana vs. Prem Prakash,Introduction: Parents play a significant role...,https://indiankanoon.org/doc/143531/,True
2,Bhola Ram Sahu vs State Of Chhattisgarh,Introduction Challenge in this appeal is to th...,http://www.indiankanoon.com,True
3,United India Fire & General Insurance Company ...,Introduction: Every contract is an agreement b...,https://indiankanoon.org/doc/59542/,True
4,State of Madhya Pradesh vs. Deepak,Introduction: In India an attempt to commit s...,https://caselaw.in/supreme-court/framing-charg...,True
...,...,...,...,...
159,State bank of India v. Santosh Gupta and Ors.,Jammu and Kashmir became a part of this nation...,https://indiankanoon.org/doc/522930/,True
160,NLSA vs. Union of India,National Legal Services Authority was formed o...,https://indiankanoon.org/doc/417020/,True
161,Mohd. Arif @ Ashfaq vs. The Reg. Supreme Court...,Our judicial system is based on principles to ...,https://indiankanoon.org/doc/1766147/,True
162,Ajay Hasia etc. Vs. Khalid Mujib Sehravardi & ...,Political Background- According to Article 12 ...,https://indiankanoon.org/doc/609139/,True


In [42]:
Final_df.to_csv('lawtimes.csv', header=True, index=False)